In [1]:
import os


In [2]:
%pwd

'c:\\Users\\KIIT0001\\Desktop\\NLProject\\Text-Summariser\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\KIIT0001\\Desktop\\NLProject\\Text-Summariser'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
#return type of function
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.textSummariser.constants import *
from src.textSummariser.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        #read both the files and return the contents of the file
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        #create directorie using config because it is box type output
        create_directories([self.config.artifacts_root])
        #it will create data ingestion folder. inside that data ingestion where it will be downloading the data
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        #creating root directory  and then performing all the below things like unzipping the data
        create_directories([config.root_dir])
        #here DataIngestionConfig is the return type as already mentioned in config.yaml
        data_ingestion_config = DataIngestionConfig(
            #below are the things it wil return
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
#for reading data from the url
import urllib.request as request
#for unzipping the dowloaded data
import zipfile
from src.textSummariser.logging import logger
from src.textSummariser.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config



    #downloading file    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                #reading the source url
                url = self.config.source_URL,
                #giving the file location
                filename = self.config.local_data_file
            )
            #logger message
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            #getting the file size
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
         

In [10]:
try:
    #calling for the classes
    config = ConfigurationManager()
    #getting the data by calling the class(right click and go to function definition)
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-12 06:19:14,680: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-12 06:19:14,686: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-12 06:19:14,689: INFO: common: created directory at: artifacts]
[2024-10-12 06:19:14,690: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-12 06:19:14,691: INFO: 2082177647: File already exists of size: ~ 7718 KB]
